In [2]:
import numpy as np

In [3]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

def init_params(layer_dims):
    params = {}
    np.random.seed(42)
    for l in range(1, len(layer_dims)):
        params[f"W{l}"] = np.random.rand(layer_dims[l], layer_dims[l-1])*0.01
        params[f"b{l}"] = np.zeros((layer_dims[l],1))
    return params

def forward_prop(X,params):
    caches = []
    A = X
    L = len(params) // 2 # this gives the number of layers as we have both weights and bias matrices that double the layer count

    for l in range(1, L+1):
        W = params[f"W{l}"]
        b = params[f"b{l}"]
        Z = W@A + b
        A = sigmoid(Z)
        caches.append((A,W,b,Z))
    return A, caches


def compute_loss(Y,A):
    m = Y.shape[1]
    loss = -np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))/m
    return np.squeeze(loss)


def back_prop(X,Y,caches):
    grads = {}
    L = len(caches)
    m = X.shape[1]
    dZ = None

    for l in reversed(range(1, L+1)):
        A,W,b,Z = caches[l-1]
        if l == L:
            dZ = A - Y # this is the output layer
        else:
            dZ = caches[l][1].T@dZ*sigmoid_derivative(Z)
        
        grads[f"dW{l}"] = dZ@caches[l-2][0].T/m if l > 1 else dZ@X.T/m
        grads[f"db{l}"] = np.sum(dZ,axis=1,keepdims=True) / m
    
    return grads

def optimise(params, grads, learning_rate):
    L = len(params) // 2
    for l in range(1, L+1):
        params[f"W{l}"] -= learning_rate*grads[f"dW{l}"]
        params[f"b{l}"] -= learning_rate*grads[f"db{l}"]
    return params

def adam_optimise(params, grads, learning_rate, beta1, beta2, epsilon, t, m, v):
    """
    Updates parameters using the Adam optimization algorithm.
    
    Parameters:
        params (dict): Dictionary containing model parameters (e.g., W1, b1, W2, b2, ...).
        grads (dict): Dictionary containing gradients of parameters (e.g., dW1, db1, ...).
        learning_rate (float): Learning rate for the optimization.
        beta1 (float): Exponential decay rate for the first moment estimates.
        beta2 (float): Exponential decay rate for the second moment estimates.
        epsilon (float): Small value to prevent division by zero.
        t (int): Timestep (iteration count).
        m (dict): Dictionary to store moving averages of gradients (first moment).
        v (dict): Dictionary to store moving averages of squared gradients (second moment).
        
    Returns:
        params (dict): Updated parameters.
        m (dict): Updated first moment estimates.
        v (dict): Updated second moment estimates.
    """
    L = len(params) // 2  # Number of layers in the neural network

    # Update parameters for each layer
    for l in range(1, L + 1):
        # Compute the moving averages of the gradients (m) and squared gradients (v)
        m[f"dW{l}"] = beta1 * m.get(f"dW{l}", 0) + (1 - beta1) * grads[f"dW{l}"]
        m[f"db{l}"] = beta1 * m.get(f"db{l}", 0) + (1 - beta1) * grads[f"db{l}"]
        v[f"dW{l}"] = beta2 * v.get(f"dW{l}", 0) + (1 - beta2) * (grads[f"dW{l}"] ** 2)
        v[f"db{l}"] = beta2 * v.get(f"db{l}", 0) + (1 - beta2) * (grads[f"db{l}"] ** 2)

        # Correct bias for first and second moments
        m_corrected_dW = m[f"dW{l}"] / (1 - beta1 ** t)
        m_corrected_db = m[f"db{l}"] / (1 - beta1 ** t)
        v_corrected_dW = v[f"dW{l}"] / (1 - beta2 ** t)
        v_corrected_db = v[f"db{l}"] / (1 - beta2 ** t)

        # Update parameters
        params[f"W{l}"] -= learning_rate * m_corrected_dW / (v_corrected_dW ** 0.5 + epsilon)
        params[f"b{l}"] -= learning_rate * m_corrected_db / (v_corrected_db ** 0.5 + epsilon)

    return params, m, v

def train(
    X,
    y,
    layer_dims,
    learning_rate=0.1,
    num_iterations=1000,
    optimizer="sgd",  # Choose between "sgd" and "adam"
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-8,
):
    """
    Trains a neural network using forward and backward propagation with an option
    to switch between normal SGD and Adam optimization.

    Parameters:
        X: Input data.
        y: Ground truth labels.
        layer_dims: List specifying the dimensions of each layer in the network.
        learning_rate: The learning rate for optimization (default: 0.1).
        num_iterations: Number of iterations for training (default: 1000).
        optimizer: Optimization algorithm to use ("sgd" or "adam").
        beta1: Exponential decay rate for the first moment estimates (Adam only).
        beta2: Exponential decay rate for the second moment estimates (Adam only).
        epsilon: Small constant to prevent division by zero (Adam only).

    Returns:
        params: Trained parameters of the network.
    """
    params = init_params(layer_dims)
    m, v = {}, {}  # Initialize Adam-specific variables
    t = 0  # Adam timestep counter

    for i in range(1, num_iterations + 1):
        # Forward propagation
        A, caches = forward_prop(X, params)

        # Compute loss
        loss = compute_loss(y, A)

        # Backward propagation
        grads = back_prop(X, y, caches)

        # Optimization step
        if optimizer == "adam":
            t += 1
            params, m, v = adam_optimise(
                params, grads, learning_rate, beta1, beta2, epsilon, t, m, v
            )
        elif optimizer == "sgd":
            params = optimise(params, grads, learning_rate)
        else:
            raise ValueError("Unsupported optimizer. Choose 'sgd' or 'adam'.")

        # Print loss every 100 iterations
        if i % 100 == 0 or i == num_iterations:
            print(f"Iteration [{i}/{num_iterations}], Loss: {loss}")

    return params


In [4]:
def gradient_check(X, Y, params, grads, layer_dims, epsilon=1e-7):
    """
    Perform gradient checking to verify the correctness of backprop gradients.

    Args:
        X (ndarray): Input data of shape (n_x, m).
        Y (ndarray): True "label" vector of shape (1, m).
        params (dict): Dictionary of your current parameters "Wl", "bl".
        grads (dict): Dictionary of your current gradients "dWl", "dbl".
        layer_dims (list): Dimensions of each layer in the network.
        epsilon (float): Small shift to compute numerical gradient.

    Returns:
        None (Prints out the differences for each parameter)
    """
    # A small helper function to compute cost given parameters
    def compute_cost_for_param_set(params):
        A, _ = forward_prop(X, params)
        cost = compute_loss(Y, A)
        return cost
    
    # Number of layers
    L = len(layer_dims) - 1  # If layer_dims = [n_x, n_h, n_y], L=2 (two sets of W, b)

    # We will store differences here
    differences = []

    for l in range(1, L+1):
        W_key = f"W{l}"
        b_key = f"b{l}"
        dW_key = f"dW{l}"
        db_key = f"db{l}"

        # ============ Gradient Check for W{l} ============ 
        w_rows, w_cols = params[W_key].shape
        for i in range(w_rows):
            for j in range(w_cols):
                # Save original value
                original_value = params[W_key][i, j]

                # J_plus
                params[W_key][i, j] = original_value + epsilon
                J_plus = compute_cost_for_param_set(params)

                # J_minus
                params[W_key][i, j] = original_value - epsilon
                J_minus = compute_cost_for_param_set(params)

                # Reset back to original value
                params[W_key][i, j] = original_value

                # Numerical approximation of the gradient
                grad_approx = (J_plus - J_minus) / (2.0 * epsilon)

                # Compare with backprop gradient
                grad_backprop = grads[dW_key][i, j]

                # Compute relative difference
                numerator = abs(grad_approx - grad_backprop)
                denominator = abs(grad_approx) + abs(grad_backprop) + 1e-12  # add small term to avoid /0
                difference = numerator / denominator
                differences.append(difference)

        # ============ Gradient Check for b{l} ============ 
        b_rows, b_cols = params[b_key].shape
        for i in range(b_rows):
            for j in range(b_cols):
                # Save original value
                original_value = params[b_key][i, j]

                # J_plus
                params[b_key][i, j] = original_value + epsilon
                J_plus = compute_cost_for_param_set(params)

                # J_minus
                params[b_key][i, j] = original_value - epsilon
                J_minus = compute_cost_for_param_set(params)

                # Reset back to original value
                params[b_key][i, j] = original_value

                # Numerical approximation of the gradient
                grad_approx = (J_plus - J_minus) / (2.0 * epsilon)

                # Compare with backprop gradient
                grad_backprop = grads[db_key][i, j]

                # Compute relative difference
                numerator = abs(grad_approx - grad_backprop)
                denominator = abs(grad_approx) + abs(grad_backprop) + 1e-12
                difference = numerator / denominator
                differences.append(difference)

    # Report results
    max_diff = np.max(differences)
    print(f"Max difference between numerical and backprop gradients: {max_diff}")
    if max_diff < 1e-7:
        print("Your backprop implementation seems to be correct!")
    else:
        print("There might be an issue with your backprop implementation.")


In [11]:
import numpy as np
import matplotlib.pyplot as plt

# Generate toy data
np.random.seed(42)
X = np.random.randn(2, 500)  # 2 features, 500 samples
y = (X[1, :] > np.sin(2 * np.pi * X[0, :])).astype(int).reshape(1, 500)  # 1 if above the sine wave, else 0
layer_dims = [2, 50, 10, 5, 1]
# After initializing params:
params = init_params(layer_dims)

# Forward prop
A, caches = forward_prop(X, params)
loss = compute_loss(y, A)

# Backward prop
grads = back_prop(X, y, caches)

# Run gradient check
gradient_check(X, y, params, grads, layer_dims, epsilon=1e-7)


Max difference between numerical and backprop gradients: 0.21347565474271563
There might be an issue with your backprop implementation.
